# Advent of Code: Day 1

--- Day 1: Sonar Sweep ---
You're minding your own business on a ship at sea when the overboard alarm goes off! You rush to see if you can help. Apparently, one of the Elves tripped and accidentally sent the sleigh keys flying into the ocean!

Before you know it, you're inside a submarine the Elves keep ready for situations like this. It's covered in Christmas lights (because of course it is), and it even has an experimental antenna that should be able to track the keys if you can boost its signal strength high enough; there's a little meter that indicates the antenna's signal strength by displaying 0-50 stars.

Your instincts tell you that in order to save Christmas, you'll need to get all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

As the submarine drops below the surface of the ocean, it automatically performs a sonar sweep of the nearby sea floor. On a small screen, the sonar sweep report (your puzzle input) appears: each line is a measurement of the sea floor depth as the sweep looks further and further away from the submarine.

For example, suppose you had the following report:

199
200
208
210
200
207
240
269
260
263
This report indicates that, scanning outward from the submarine, the sonar sweep found depths of 199, 200, 208, 210, and so on.

The first order of business is to figure out how quickly the depth increases, just so you know what you're dealing with - you never know if the keys will get carried into deeper water by an ocean current or a fish or something.

To do this, count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.) In the example above, the changes are as follows:

199 (N/A - no previous measurement)
200 (increased)
208 (increased)
210 (increased)
200 (decreased)
207 (increased)
240 (increased)
269 (increased)
260 (decreased)
263 (increased)
In this example, there are 7 measurements that are larger than the previous measurement.

How many measurements are larger than the previous measurement?

--- Part Two ---
Considering every single measurement isn't as useful as you expected: there's just too much noise in the data.

Instead, consider sums of a three-measurement sliding window. Again considering the above example:

199  A      
200  A B    
208  A B C  
210    B C D
200  E   C D
207  E F   D
240  E F G  
269    F G H
260      G H
263        H
Start by comparing the first and second three-measurement windows. The measurements in the first window are marked A (199, 200, 208); their sum is 199 + 200 + 208 = 607. The second window is marked B (200, 208, 210); its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.

Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare A with B, then compare B with C, then C with D, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.

In the above example, the sum of each three-measurement window is as follows:

A: 607 (N/A - no previous sum)
B: 618 (increased)
C: 618 (no change)
D: 617 (decreased)
E: 647 (increased)
F: 716 (increased)
G: 769 (increased)
H: 792 (increased)
In this example, there are 5 sums that are larger than the previous sum.

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

### Using text file and looping through a list

In [5]:
def depth_input(input_file:str):
    """Build array from single column text file. Convert to integers"""
    depths = []
    
    with open(input_file, 'r') as file:
        for line in file:
            try:
                depths.append(int(line.strip()))
            except ValueError as e: 
                    pass
                
    return depths


def count_increases(depths:list) -> int:
    """Returns the number of increases between elements in a list """
    increase = 0
    for i in range(len(depths)):
        if i == 0:
            pass
        elif depths[i] > depths[i-1]:
            increase += 1
    return increase


def count_window_increases(depths:list, window:int) -> int:
    """ Reads the number of increases between rolling window 
    """
    increase = 0
    # skipping first two elements as can't form full 3 element windows - using subraction on index
    for i in range(window, (len(depths))):
        current_window = sum(depths[i-window:i])
        next_window = sum(depths[i-window+1:i+1])
        if current_window < next_window:
            increase += 1
    return increase

In [6]:
# Workflow

depths = depth_input('data/sonar_depths.txt')
increases = count_increases(depths)
window_increases = count_window_increases(depths,3)

print(f'Number of increases between readings: {increases}')
print(f'Number of increases between rolling windows: {window_increases}')

Number of increases between readings: 1482
Number of increases between rolling windows: 1518


### Using NumPy rather than list and loops

In [2]:
import numpy as np

def np_input(input_file:str) -> np.array: 
    """Build array of integers from single column text file. Load through numpy"""
    with open(input_file, 'r') as file:
        depths = np.loadtxt(file, dtype=np.int16)
    return depths


def np_change(depths:np.array) -> np.array:
    """
    Returns the difference between an element and the following in an array.  Length is one less than input 
    """
    change = depths[1:] - depths[:-1]
    return change


def np_3_window(depths:np.array) -> np.array:
    """Static window of three elements."""
    summed_window = depths[:-2] + depths[1:-1] + depths[2:]
    return summed_window


def np_window(depths:np.array, window:int) -> np.array:
    """
    Returns a sum of all elements using a window size.
    Loops through size of window and sums resulting array slices.  
    If selecting a window of 1 it will return the original array.
    
    Note: needs to copy slice otherwise adds to original array
    """
    # First element up to window length
    end_idx = (1-window)   if not window == 1 else None
    summed_window = np.copy(depths[:end_idx])
    # Add each subsequent elements for the window sum
    for i in range(1, window):
        start_idx = (0 + i)
        end_idx =   (1 + i - window)   if not (i - window == -1) else None
        summed_window += depths[start_idx:end_idx]
    return summed_window


def sum_of_increases(array:np.array) -> int:
    """Returns the number of positive values in an array"""
    return sum(array > 0)

In [3]:
# Workflow

np_depths = np_input('data/sonar_depths.txt')

single_increases = sum_of_increases(np_change(np_depths))

window_increases = sum_of_increases(np_change(np_window(np_depths,3)))

print(f'Number of increases between readings: {single_increases}')
print(f'Number of increases between rolling 3 element windows: {window_increases}')

Number of increases between readings: 1482
Number of increases between rolling 3 element windows: 1518
